In [1]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation, rc
from IPython.display import HTML
from plotly.offline import init_notebook_mode, iplot

from exoexplorations import *

In [8]:
import pandas as pd
imsize = 400, 400
# adjust the planet and the star's radius
# make star less than 0.5 the smallest imsize 
pr, sr = 75, 100
time, flux = lightcurve(sr, pr, i=50, imsize=imsize)
dfs = pd.DataFrame({'time': time, 'flux': flux, 'star_radius': sr, 'planet_radius': pr})

In [9]:
first_dip = np.where(np.diff(flux) < 0)[0][0]
first_full = np.where(np.diff(flux) < 0)[0][-1]
sec_dip = np.where(np.diff(flux) > 0)[0][0]
sec_full = np.where(np.diff(flux) > 0)[0][-1]

In [11]:
import plotly.express as px
import plotly.graph_objects as go
import plotly

from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

fig = go.Figure(
    data=[go.Scatter(x=dfs.time.values, y=dfs.flux.values,
                     mode="lines",
                     name="Imaginary Planet",
                     line=dict(width=2, color="blue")),
          go.Scatter(x=dfs.time.values, y=dfs.flux.values,
                     mode="lines",
                     name="Light from a Star",
                     line=dict(width=2, color="blue"))],
    layout=go.Layout(
        xaxis=dict(range=[0, imsize[0]], autorange=False, zeroline=False),
        yaxis=dict(range=[0, 1.3], autorange=False, zeroline=False),
        title_text="Exoplanet Transit", hovermode="closest",
        xaxis_title='Time',
        yaxis_title='Flux',
        updatemenus=[dict(type="buttons",
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None])])]),
    frames=[go.Frame(
        data=[go.Scatter(
            x=[dfs.time.values[::5][k]],
            y=[dfs.flux.values[::5][k]],
            mode="markers",
            marker=dict(color="red", size=10))])

        for k in range(dfs.time.values[::5].shape[0])]
)

fig.update_layout(
    showlegend=False,
    annotations=[
        dict(
            x=time[first_dip],
            y=flux[first_dip],
            xref="x",
            yref="y",
            text="Planet starts to occult the star",
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=-40
        ),        
        dict(
            x=time[first_full],
            y=flux[first_full],
            xref="x",
            yref="y",
            text="Planet is fully in front of the star",
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=-40
        ), 
        dict(
            x=time[sec_dip],
            y=flux[sec_dip],
            xref="x",
            yref="y",
            text="Planet has reached the edge of the star",
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=40,
            
        ),
        dict(
            x=time[sec_full],
            y=flux[sec_full],
            xref="x",
            yref="y",
            text="Planet has stopped occulting the star",
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=-40
        )
    ]
)

fig